In [12]:
# ! pip install --upgrade ultralytics
# ! pip install ftfy
# ! pip install regex
# ! pip install iopath

In [2]:
import ultralytics
ultralytics.__version__

'8.3.237'

In [12]:
from ultralytics import SAM

model = SAM('sam2.1_s.pt')

# Segment with point prompt
results = model("image1000.jpg", points=[150, 150], labels=[1])


image 1/1 c:\Users\\Documents\HYPERSPECTRAL CAMERA\Rapeseed_drought_03.12.2025\image1000.jpg: 1024x1024 1 0, 1407.8ms
Speed: 13.2ms preprocess, 1407.8ms inference, 18.9ms postprocess per image at shape (1, 3, 1024, 1024)


In [3]:
import os
import re
from pathlib import Path

import cv2
import numpy as np
from ultralytics import SAM

# ----------------- CONFIG -----------------
FOLDER_LIST_FILE = "folder_list.txt"
SKIP_FIRST_N_FOLDERS = 3

SPECTRAL_SUBFOLDER = "Spectral_Cube"
MASKS_SUBFOLDER = "masks"

MODEL_PATH = "sam2.1_s.pt"
SCALE = 0.25

TARGET_IMAGE_NUM = 700  # start with image700 if exists, else nearest

# display style
ALPHA = 0.45
MASK_COLOR = (0, 255, 0)  # green (BGR)

# ----------------- KEY CODES (cross-platform-ish) -----------------
# cv2.waitKeyEx() is best for arrows. These are common on Windows.
KEY_LEFT_WIN  = 2424832
KEY_RIGHT_WIN = 2555904
KEY_UP_WIN    = 2490368
KEY_DOWN_WIN  = 2621440

# Some builds return these low codes for arrows (often Linux / X11).
KEY_LEFT_LOW  = 81
KEY_RIGHT_LOW = 83
KEY_UP_LOW    = 82
KEY_DOWN_LOW  = 84

KEY_ESC = 27
KEY_ENTER_1 = 13
KEY_ENTER_2 = 10

# ----------------- REGEX -----------------
IMG_RE = re.compile(r"^image(\d{3,4})\.(jpg|jpeg|png)$", re.IGNORECASE)

# ----------------- HELPERS -----------------
def resolve_corrected_folder(folder: Path):
    """
    If 'Corrected_<foldername>' exists in the same parent directory,
    return that path; otherwise return the original folder.
    """
    corrected = folder.parent / f"Corrected_{folder.name}"
    if corrected.is_dir():
        return corrected
    return folder

def resize_image(img, scale: float):
    h, w = img.shape[:2]
    new_size = (max(1, int(w * scale)), max(1, int(h * scale)))
    return cv2.resize(img, new_size, interpolation=cv2.INTER_AREA)

def overlay_union_mask(img_bgr, union01):
    out = img_bgr.copy()
    if union01 is None:
        return out

    out[union01 == 1] = (
        (1 - ALPHA) * out[union01 == 1] + ALPHA * np.array(MASK_COLOR)
    ).astype(np.uint8)

    mask_255 = (union01 * 255).astype(np.uint8)
    contours, _ = cv2.findContours(mask_255, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    cv2.drawContours(out, contours, -1, (0, 0, 255), 2)
    return out

def draw_points(img_bgr, points, labels):
    out = img_bgr.copy()
    for (x, y), lab in zip(points, labels):
        # label 1: positive (white dot), label 0: negative (gray dot)
        fill = (255, 255, 255) if lab == 1 else (180, 180, 180)
        edge = (0, 0, 0)
        cv2.circle(out, (x, y), 4, fill, -1)
        cv2.circle(out, (x, y), 8, edge, 2)
    return out

def get_first_mask01(result0):
    if result0.masks is None:
        return None
    m = result0.masks.data  # (N,H,W)
    try:
        m_np = m.cpu().numpy()
    except Exception:
        m_np = np.array(m)

    if m_np.ndim != 3 or m_np.shape[0] < 1:
        return None
    return (m_np[0] > 0.5).astype(np.uint8)

def read_folder_list(path: str, skip_n: int):
    p = Path(path)
    if not p.exists():
        raise FileNotFoundError(f"Missing {path}")

    folders = []
    for line in p.read_text(encoding="utf-8", errors="ignore").splitlines():
        line = line.strip()
        if not line:
            continue
        folders.append(Path(os.path.expanduser(line)).resolve())

    if len(folders) <= skip_n:
        return []
    return folders[skip_n:]

def list_spectral_images(folder: Path):
    spectral = folder / SPECTRAL_SUBFOLDER
    if not spectral.is_dir():
        return []  # no Spectral_Cube

    items = []
    for entry in spectral.iterdir():
        if not entry.is_file():
            continue
        m = IMG_RE.match(entry.name)
        if not m:
            continue
        num = int(m.group(1))
        items.append((num, entry))

    items.sort(key=lambda t: t[0])
    return items  # list of (num, Path)

def choose_start_index(items, target_num: int):
    if not items:
        return 0
    nums = [n for n, _ in items]
    # exact hit preferred
    if target_num in nums:
        return nums.index(target_num)
    # nearest
    return min(range(len(nums)), key=lambda i: abs(nums[i] - target_num))

def next_available_mask_path(masks_dir: Path):
    masks_dir.mkdir(parents=True, exist_ok=True)
    x = 1
    while True:
        candidate = masks_dir / f"masks{x}.jpg"
        if not candidate.exists():
            return candidate
        x += 1

# ----------------- GLOBAL STATE for mouse -----------------
pending_click = None  # (x,y,label)

def on_mouse(event, x, y, flags, param):
    global pending_click
    # Left click -> positive point
    if event == cv2.EVENT_LBUTTONDOWN:
        pending_click = (x, y, 1)
    # Right click -> negative point (optional but useful)
    elif event == cv2.EVENT_RBUTTONDOWN:
        pending_click = (x, y, 0)

# ----------------- MAIN APP -----------------
def main():
    global pending_click

    raw_folders = read_folder_list(FOLDER_LIST_FILE, SKIP_FIRST_N_FOLDERS)
    folders = [resolve_corrected_folder(f) for f in raw_folders]

    if not folders:
        raise RuntimeError("No folders to process (after skipping first 3 lines).")

    model = SAM(MODEL_PATH)

    win = "SAM Folder/Image Navigator (25%)"
    cv2.namedWindow(win, cv2.WINDOW_NORMAL)
    cv2.setMouseCallback(win, on_mouse)

    folder_idx = 0

    # Per-folder/per-image state
    items = []           # list of (num, path)
    img_idx = 0
    img_resized = None
    img_num = None
    img_path = None

    # Per-object mask build state (cleared after saving)
    points = []
    labels = []
    submasks = []  # list of (H,W) uint8 0/1
    warn_top_frames = 0
    warn_bottom_frames = 0

    def load_current_folder():
        nonlocal items, img_idx
        nonlocal img_resized, img_num, img_path
        nonlocal points, labels, submasks

        folder = folders[folder_idx]
        items = list_spectral_images(folder)
        if not items:
            img_idx = 0
            img_resized = None
            img_num = None
            img_path = None
            points.clear(); labels.clear(); submasks.clear()
            return

        img_idx = choose_start_index(items, TARGET_IMAGE_NUM)
        load_current_image(reset_object=True)

    def load_current_image(reset_object: bool):
        nonlocal img_resized, img_num, img_path
        nonlocal points, labels, submasks

        if not items:
            img_resized = None
            img_num = None
            img_path = None
            return

        img_num, img_path = items[img_idx]
        img_full = cv2.imread(str(img_path))
        if img_full is None:
            img_resized = None
            return

        img_resized = resize_image(img_full, SCALE)

        if reset_object:
            points.clear()
            labels.clear()
            submasks.clear()

    # initial load
    load_current_folder()

    print("Controls:")
    print("  Left click  : add POS sub-mask (label=1) from that single point")
    print("  Right click : add NEG sub-mask (label=0) from that single point")
    print("  Left/Right  : previous/next image number in Spectral_Cube")
    print("  Up/Down     : previous/next folder in folder_list.txt (after skipping first 3)")
    print("  e           : undo last point + last sub-mask")
    print("  Enter       : save union mask to <folder>/masks/masksX.jpg (no overwrite), then clear points/masks")
    print("  q / Esc     : quit")

    while True:
        folder = folders[folder_idx]

        # compute union
        if len(submasks) == 0:
            union01 = None
        else:
            union01 = np.clip(np.sum(submasks, axis=0), 0, 1).astype(np.uint8)

        # build view
        if img_resized is None:
            view = np.zeros((480, 900, 3), dtype=np.uint8)
            cv2.putText(view, "No image loaded (missing Spectral_Cube or unreadable images).",
                        (20, 60), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 255, 255), 2)
        else:
            view = overlay_union_mask(img_resized, union01)
            view = draw_points(view, points, labels)

        # UI overlays
        header = f"Folder {folder_idx+1}/{len(folders)}: {folder}"
        cv2.putText(view, header[:120], (10, 25),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.55, (255, 255, 255), 2)

        if img_path is not None:
            info = f"Image: image{img_num}  ({img_idx+1}/{len(items)})  |  Sub-masks: {len(submasks)}"
        else:
            info = "Image: (none)"
        cv2.putText(view, info, (10, 50),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.55, (255, 255, 255), 2)

        cv2.putText(view, "Arrows: img/folder | click: add | e: undo | Enter: save+clear | q/Esc: quit",
                    (10, 75), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2)

        # warnings
        if warn_top_frames > 0:
            cv2.putText(view, "FIRST FOLDER (can't go up)", (10, 110),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
            warn_top_frames -= 1
        if warn_bottom_frames > 0:
            cv2.putText(view, "LAST FOLDER (can't go down)", (10, 110),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
            warn_bottom_frames -= 1

        cv2.imshow(win, view)

        # handle mouse click -> segment immediately (single-point SAM)
        if pending_click is not None and img_resized is not None:
            x, y, lab = pending_click
            pending_click = None

            H, W = img_resized.shape[:2]
            results = model(img_resized, points=[x, y], labels=[lab])
            mask01 = get_first_mask01(results[0])

            if mask01 is None or mask01.shape != (H, W):
                print("No mask returned for that point (or shape mismatch). Try another point.")
            else:
                points.append([x, y])
                labels.append(lab)
                submasks.append(mask01)
                print(f"Added sub-mask #{len(submasks)} at point ({x},{y}) label={lab} in image{img_num}.")

        key = cv2.waitKeyEx(20)

        # quit
        if key in (KEY_ESC, ord('q')):
            break

        # undo
        if key == ord('e'):
            if points: points.pop()
            if labels: labels.pop()
            if submasks: submasks.pop()
            print(f"Undo. Remaining sub-masks: {len(submasks)}")

        # save union on Enter, then clear points/masks (start next object)
        if key in (KEY_ENTER_1, KEY_ENTER_2):
            if img_resized is None or img_path is None:
                print("No image loaded; nothing to save.")
                continue
            if not submasks:
                print("Nothing to save (no sub-masks).")
                continue

            union01 = np.clip(np.sum(submasks, axis=0), 0, 1).astype(np.uint8)
            mask_to_save = (union01 * 255).astype(np.uint8)

            masks_dir = folder / MASKS_SUBFOLDER
            save_path = next_available_mask_path(masks_dir)

            ok = cv2.imwrite(str(save_path), mask_to_save)
            if ok:
                print(f"Saved mask to: {save_path}")
                # clear for next object
                points.clear()
                labels.clear()
                submasks.clear()
            else:
                print("Failed to save mask.")

        # image navigation (left/right)
        if key in (KEY_LEFT_WIN, KEY_LEFT_LOW):
            if items and img_idx > 0:
                img_idx -= 1
                load_current_image(reset_object=True)
                print(f"Switched to image{items[img_idx][0]} (cleared points/masks).")
        elif key in (KEY_RIGHT_WIN, KEY_RIGHT_LOW):
            if items and img_idx < len(items) - 1:
                img_idx += 1
                load_current_image(reset_object=True)
                print(f"Switched to image{items[img_idx][0]} (cleared points/masks).")

        # folder navigation (up/down)
        if key in (KEY_UP_WIN, KEY_UP_LOW):
            if folder_idx == 0:
                warn_top_frames = 30  # show warning ~30 frames
            else:
                folder_idx -= 1
                warn_top_frames = 0
                warn_bottom_frames = 0
                load_current_folder()
                print(f"Moved to folder {folder_idx+1}/{len(folders)}.")
        elif key in (KEY_DOWN_WIN, KEY_DOWN_LOW):
            if folder_idx == len(folders) - 1:
                warn_bottom_frames = 30
            else:
                folder_idx += 1
                warn_top_frames = 0
                warn_bottom_frames = 0
                load_current_folder()
                print(f"Moved to folder {folder_idx+1}/{len(folders)}.")

    cv2.destroyAllWindows()

if __name__ == "__main__":
    main()


Controls:
  Left click  : add POS sub-mask (label=1) from that single point
  Right click : add NEG sub-mask (label=0) from that single point
  Left/Right  : previous/next image number in Spectral_Cube
  Up/Down     : previous/next folder in folder_list.txt (after skipping first 3)
  e           : undo last point + last sub-mask
  Enter       : save union mask to <folder>/masks/masksX.jpg (no overwrite), then clear points/masks
  q / Esc     : quit
Switched to image710 (cleared points/masks).
Switched to image720 (cleared points/masks).
Switched to image710 (cleared points/masks).
Switched to image720 (cleared points/masks).
Switched to image730 (cleared points/masks).
Switched to image740 (cleared points/masks).
Switched to image750 (cleared points/masks).
Switched to image760 (cleared points/masks).
Switched to image770 (cleared points/masks).
Switched to image780 (cleared points/masks).
Switched to image790 (cleared points/masks).
Switched to image800 (cleared points/masks).
Switche

In [4]:
import re
from pathlib import Path

import cv2
import numpy as np
import pandas as pd

# ----------------- CONFIG -----------------
FOLDER_LIST_FILE = "folder_list.txt"
SKIP_FIRST_N_FOLDERS = 3

SPECTRAL_SUBFOLDER = "Spectral_Cube"
MASKS_SUBFOLDER = "masks"

OUTPUT_XLSX = "spectral_data.xlsx"

# Wavelength list: 365, then 400..1000 step 5
WAVELENGTHS = [365] + list(range(400, 1001, 5))

# Spectral image naming pattern
IMG_EXTS = (".jpg", ".jpeg", ".png")


# ----------------- HELPERS -----------------
def resolve_corrected_folder(folder: Path) -> Path:
    """Use Corrected_<foldername> if it exists as a sibling folder; else use folder."""
    corrected = folder.parent / f"Corrected_{folder.name}"
    return corrected if corrected.is_dir() else folder


def read_folder_list(path: str, skip_n: int) -> list[Path]:
    p = Path(path)
    if not p.exists():
        raise FileNotFoundError(f"Missing {path}")

    folders = []
    for line in p.read_text(encoding="utf-8", errors="ignore").splitlines():
        line = line.strip()
        if not line:
            continue
        folders.append(Path(line).expanduser().resolve())

    return folders[skip_n:] if len(folders) > skip_n else []


def find_wavelength_image(spectral_dir: Path, wl: int) -> Path | None:
    """Return path to image{wl}.(jpg/png/...) if present (any supported extension)."""
    base = f"image{wl}"
    for ext in IMG_EXTS:
        candidate = spectral_dir / f"{base}{ext}"
        if candidate.exists():
            return candidate
        # also try uppercase extensions that might exist
        candidate2 = spectral_dir / f"{base}{ext.upper()}"
        if candidate2.exists():
            return candidate2
    # if not exact match, not found
    return None


def list_masks(masks_dir: Path) -> list[Path]:
    """List mask files (masksX.jpg/png or maskX.jpg/png) sorted by their numeric suffix if present."""
    if not masks_dir.is_dir():
        return []

    rx = re.compile(r"^(masks|mask)(\d+)\.(jpg|jpeg|png)$", re.IGNORECASE)
    items = []
    for f in masks_dir.iterdir():
        if not f.is_file():
            continue
        m = rx.match(f.name)
        if not m:
            continue
        idx = int(m.group(2))
        items.append((idx, f))

    items.sort(key=lambda t: t[0])
    return [p for _, p in items]


def load_grayscale(path: Path) -> np.ndarray:
    """
    Always return a 2D uint8 grayscale image.
    Handles 2D, (H,W,1), BGR, BGRA.
    """
    img = cv2.imread(str(path), cv2.IMREAD_UNCHANGED)
    if img is None:
        raise RuntimeError(f"Could not read image: {path}")

    # Already 2D grayscale
    if img.ndim == 2:
        return img.astype(np.uint8)

    # 3D but possibly 1-channel
    if img.ndim == 3:
        ch = img.shape[2]

        # (H,W,1) -> squeeze to (H,W)
        if ch == 1:
            return img[:, :, 0].astype(np.uint8)

        # (H,W,4) BGRA -> BGR
        if ch == 4:
            img = cv2.cvtColor(img, cv2.COLOR_BGRA2BGR)
            ch = 3

        # (H,W,3) BGR -> GRAY
        if ch == 3:
            img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
            return img.astype(np.uint8)

    # If we got here, it’s some unexpected format
    raise ValueError(f"Unsupported image shape {img.shape} for file: {path}")



def resize_mask_to_image(mask_small: np.ndarray, target_h: int, target_w: int) -> np.ndarray:
    # force to 2D first
    if mask_small.ndim == 3:
        ch = mask_small.shape[2]
        if ch == 1:
            mask_small = mask_small[:, :, 0]
        elif ch == 4:
            mask_small = cv2.cvtColor(mask_small, cv2.COLOR_BGRA2BGR)
            mask_small = cv2.cvtColor(mask_small, cv2.COLOR_BGR2GRAY)
        elif ch == 3:
            mask_small = cv2.cvtColor(mask_small, cv2.COLOR_BGR2GRAY)
        else:
            raise ValueError(f"Unsupported mask shape {mask_small.shape}")

    mask_resized = cv2.resize(mask_small, (target_w, target_h), interpolation=cv2.INTER_NEAREST)
    return (mask_resized > 0).astype(np.uint8)


def masked_mean_sd(gray_img: np.ndarray, mask01: np.ndarray) -> tuple[float, float] | tuple[None, None]:
    # Ensure 2D
    if gray_img.ndim == 3:
        gray_img = cv2.cvtColor(gray_img, cv2.COLOR_BGR2GRAY)

    if mask01.ndim == 3:
        mask01 = cv2.cvtColor(mask01, cv2.COLOR_BGR2GRAY)
        mask01 = (mask01 > 0).astype(np.uint8)

    # If shape mismatch, fix it (don’t crash)
    if gray_img.shape != mask01.shape:
        mask01 = cv2.resize(mask01.astype(np.uint8), (gray_img.shape[1], gray_img.shape[0]),
                            interpolation=cv2.INTER_NEAREST)
        mask01 = (mask01 > 0).astype(np.uint8)

    vals = gray_img[mask01 == 1].astype(np.float64)
    if vals.size == 0:
        return None, None

    mean = float(vals.mean())
    sd = float(vals.std(ddof=0))
    return mean, sd


# ----------------- MAIN -----------------
def main():
    raw_folders = read_folder_list(FOLDER_LIST_FILE, SKIP_FIRST_N_FOLDERS)

    # store tuples: (original_name, resolved_path)
    folders = []
    for f in raw_folders:
        original_name = f.name              # exactly as in folder_list.txt
        resolved_path = resolve_corrected_folder(f)
        folders.append((original_name, resolved_path))

    if not folders:
        raise RuntimeError("No folders to process (after skipping first 3 lines).")

    rows = []

    # predefine columns we want
    mean_cols = [f"mean_{wl}" for wl in WAVELENGTHS]
    sd_cols = [f"sd_{wl}" for wl in WAVELENGTHS]

    for folder_name, folder_path in folders:
        spectral_dir = folder_path / SPECTRAL_SUBFOLDER
        masks_dir = folder_path / MASKS_SUBFOLDER


        if not spectral_dir.is_dir():
            # still create rows? usually skip
            continue

        mask_files = list_masks(masks_dir)
        if not mask_files:
            continue

        # For each mask in this folder
        for mask_path in mask_files:
            mask_small = load_grayscale(mask_path)

            row = {"folder": folder_name, "mask": mask_path.name}

            # compute stats for each wavelength
            for wl in WAVELENGTHS:
                img_path = find_wavelength_image(spectral_dir, wl)
                if img_path is None:
                    row[f"mean_{wl}"] = np.nan
                    row[f"sd_{wl}"] = np.nan
                    continue

                gray = load_grayscale(img_path)
                H, W = gray.shape[:2]
                mask01 = resize_mask_to_image(mask_small, H, W)

                mean, sd = masked_mean_sd(gray, mask01)
                row[f"mean_{wl}"] = np.nan if mean is None else mean
                row[f"sd_{wl}"] = np.nan if sd is None else sd

            rows.append(row)

    # Build dataframe with stable column order
    df = pd.DataFrame(rows, columns=["folder", "mask"] + mean_cols + sd_cols)

    # Drop columns that are completely NA (excluding folder/mask)
    keep_base = ["folder", "mask"]
    data_cols = [c for c in df.columns if c not in keep_base]
    df_clean = pd.concat([df[keep_base], df[data_cols].dropna(axis=1, how="all")], axis=1)

    # Convert remaining NaNs to 'NA' strings as requested
    df_clean = df_clean.fillna("NA")

    # Save
    df_clean.to_excel(OUTPUT_XLSX, index=False)
    print(f"Saved: {OUTPUT_XLSX}  (rows={len(df_clean)}, cols={len(df_clean.columns)})")


if __name__ == "__main__":
    main()


Saved: spectral_data.xlsx  (rows=33, cols=124)


In [5]:
import re
import pandas as pd

INPUT_XLSX = "spectral_data.xlsx"
OUTPUT_XLSX = "spectral_data_reshaped.xlsx"

df = pd.read_excel(INPUT_XLSX)

base_cols = [c for c in ["folder", "mask"] if c in df.columns]

# Pick mean_* and sd_* columns
mean_cols = [c for c in df.columns if re.fullmatch(r"mean_\d+", str(c))]
sd_cols   = [c for c in df.columns if re.fullmatch(r"sd_\d+", str(c))]

# Build mean and sd dataframes
df_mean = df[base_cols + mean_cols].copy()
df_sd   = df[base_cols + sd_cols].copy()

# Rename columns: mean_365 -> 365, sd_365 -> 365 (as strings)
df_mean.rename(columns={c: c.split("_", 1)[1] for c in mean_cols}, inplace=True)
df_sd.rename(columns={c: c.split("_", 1)[1] for c in sd_cols}, inplace=True)

# Optional: sort wavelength columns numerically (keeps folder/mask first)
def sort_wavelength_cols(d):
    wl_cols = [c for c in d.columns if c not in base_cols]
    wl_cols_sorted = sorted(wl_cols, key=lambda x: int(x))
    return d[base_cols + wl_cols_sorted]

df_mean = sort_wavelength_cols(df_mean)
df_sd = sort_wavelength_cols(df_sd)

# Save to two sheets
with pd.ExcelWriter(OUTPUT_XLSX, engine="openpyxl") as writer:
    df_mean.to_excel(writer, sheet_name="mean", index=False)
    df_sd.to_excel(writer, sheet_name="sd", index=False)

print(f"Saved reshaped Excel to: {OUTPUT_XLSX}")


Saved reshaped Excel to: spectral_data_reshaped.xlsx
